In [31]:
#day23
import heapq
from typing import Optional

VALUE = [0] * 70  # ord('D') = 68
for c, v in zip('ABCD', [1, 10, 100, 1000]):
    VALUE[ord(c)] = v

PART = 2 # either 1 or 2
# PROBLEM_P = b'BACDBCDA       '  # test in problem description
PROBLEM_P = b'DDAACBCB       ' # xdavidliu actual input

def InsertPart2(p: bytes):
    return p[0:1] + b'DD' + p[1:3] + b'CB' + p[3:5] + b'BA' + p[5:7] + b'AC' + p[7:]

assert PART in (1, 2)
if PART == 2:
    PROBLEM_P = InsertPart2(PROBLEM_P)

H4 = len(PROBLEM_P) - 7
H = H4 // 4
SPACE = ord(' ')

PARTIAL = [None] * 70
for c in 'ABCD':
    PARTIAL[ord(c)] = [bytes(' ' * (H - n) + c * n, 'utf') for n in range(H)]

DIST = [[None] * len(PROBLEM_P) for _ in PROBLEM_P]
for r0 in range(4):
    for r1 in range(4):
        if r0 >= r1: continue  # within room dist not needed; also r0 > r1 handled by symmetry
        for k0 in range(H):
            for k1 in range(H):
                # Starting from i0, take k0+1 to enter hallway, then 2*(r1-r0) to reach
                # spot in hallway right outside r1, then k1+1 to reach i1.
                i0 = r0 * H + k0
                i1 = r1 * H + k1
                DIST[i0][i1] = DIST[i1][i0] = 2*(r1 - r0) + k0 + 1 + k1 + 1
for h in range(7):
    for r in range(4):
        for k in range(H):
            # start from ir, take k+1 to enter hallway, then abs(2*r - 2*h + 3) to ih.
            # Use diagram in RightOfRoom below to convince yourself of this.
            # ALSO! the first and last h are special, so subtract 1!
            ir = r * H + k
            ih = H4 + h
            val = k + 1 + abs(2*r - 2*h + 3)
            if h in (0, 6): val -= 1
            DIST[ir][ih] = DIST[ih][ir] = val

def RightOfRoom(r: int) -> int:
    # h = 0 1 2 3 4 5 6
    # r =    0 1 2 3
    return H4 + r + 2

def ClearBetweenRoom(p: bytes, r0: int, r1: int) -> bool:
    assert r0 != r1
    if r0 > r1: r0, r1 = r1, r0
    return p[RightOfRoom(r0) : RightOfRoom(r1)].isspace()

# h itself need not be space
def ClearHallwayToRoom(p: bytes, h: int, r: int) -> bool:
    rr = RightOfRoom(r)
    sl = slice(H4 + h + 1, rr) if h < r + 2 else slice(rr, H4 + h)
    psl = p[sl]
    # When the index of h in p coincides with rr, psl will be empty
    return (not psl) or psl.isspace()

def IsPartial(p: bytes, r: int) -> bool:
    return p[r*H : (r+1)*H] in PARTIAL[r + ord('A')]

def LowestSpace(p: bytes, r: int) -> int:
    i = r * H
    assert p[i] == SPACE
    nxt = (r+1)*H
    while i + 1 < nxt and p[i+1] == SPACE: i += 1
    return i

def Top(p: bytes, r: int) -> int:
    i = r * H
    while p[i] == SPACE: i += 1
    assert i < (r+1)*H
    return i

def Drop(p: bytearray, h: int, r: int) -> Optional[int]:
    ih = H4 + h
    ir = LowestSpace(p, r)
    c = p[ih]
    p[ih], p[ir] = p[ir], p[ih]
    return VALUE[c] * DIST[ih][ir]

DROP_MEMO = dict()

def DropUntilStop(p: bytearray) -> int:
    p0 = bytes(p)
    res = DROP_MEMO.get(p0)
    if res is not None:
        val, pf = res
        p.clear()
        p.extend(pf)
        return val
    is_partial = [IsPartial(p, r) for r in range(4)]
    improved = True
    val = 0
    while improved:
        improved = False
        if not p[H4:].isspace():
            for h in range(7):
                c = p[H4 + h]
                if c == SPACE: continue
                r = c - ord('A')
                if is_partial[r] and ClearHallwayToRoom(p, h, r):
                    val += Drop(p, h, r)
                    improved = True
        if not all(is_partial):
            for r in range(4):
                if is_partial[r]: continue
                ir = Top(p, r)
                c = p[ir]
                rd = c - ord('A')
                if not (is_partial[rd] and ClearBetweenRoom(p, r, rd)): continue
                ird = LowestSpace(p, rd)  # destination
                p[ir], p[ird] = p[ird], p[ir]
                val += VALUE[c] * DIST[ir][ird]
                improved = True
                is_partial[r] = IsPartial(p, r)  # The source room may become partial.
    DROP_MEMO[p0] = (val, bytes(p))
    return val

def Dijkstra(p0: bytes) -> int:
    hp = [(0, p0)]
    best = dict()
    DONE = bytes('A' * H + 'B' * H + 'C' * H + 'D' * H + ' ' * 7, 'utf')
    while hp:
        cost, p = heapq.heappop(hp)
        # invariant: everything popped cannot be further DroppedUntilStop
        if p == DONE: return cost
        b = best.get(p)
        if b is not None:
            assert cost >= b  # cannot be < b because it must've been previously popped
            continue
        best[p] = cost
        for r in range(4):
            if IsPartial(p, r): continue
            ir = Top(p, r)
            c = p[ir]
            for h in range(7):
                ih = H4 + h
                if not (p[ih] == SPACE and ClearHallwayToRoom(p, h, r)): continue
                pa = bytearray(p)
                pa[ir], pa[ih] = pa[ih], pa[ir]
                new_cost = cost + VALUE[c] * DIST[ir][ih] + DropUntilStop(pa)
                pp = bytes(pa)
                bb = best.get(pp)
                if bb is not None:
                    assert new_cost >= bb
                else:
                    heapq.heappush(hp, (new_cost, pp))

# For debugging purposes; not actually needed
def Draw(p: bytes) -> None:
   """ex: Draw('BACDBCDA.......')"""
   p = p.decode('utf').replace(' ', '.')
   print('#############')
   hall = p[H4:]
   draw_hall = hall[0] + '.'.join(hall[1:-1]) + hall[-1]
   print(f'#{draw_hall}#')
   for i in range(H):
       x = '#'.join(p[i:H4:H])
       side = '##' if i == 0 else '  '
       print(f'{side}#{x}#{side}')
   print('  #########  ')

print(f'part {PART} = {Dijkstra(PROBLEM_P)}')


part 2 = 47484


In [29]:
#day22
import re
from collections import namedtuple
from operator import le as less_than_or_equal
from time import perf_counter

input_regex = re.compile(r"(?m)(on|off) x=(-?\d+)\.\.(-?\d+),y=(-?\d+)..(-?\d+),z=(-?\d+)..(-?\d+)")
Instruction = namedtuple("Instruction", "turn cuboid")
Coordinate = namedtuple("Coordinate", "x y z")
Cuboid = namedtuple("Cuboid", "min max")
instructions:list[Instruction] = []

with open("input.txt") as file:
    for line in file:
        turn, x1, x2, y1, y2, z1, z2 = input_regex.match(line).groups()
        start = Coordinate(int(x1), int(y1), int(z1))
        end   = Coordinate(int(x2), int(y2), int(z2))
        my_cuboid = Cuboid(start, end)
        instructions.append(
            Instruction(turn, my_cuboid)
        )

start_time = perf_counter()

core = []   # List to store the blocks that are on

# Loop through all instructions
for turn, new in instructions:
    new:Cuboid      # The region of the instruction
    new_core = []   # List of blocks that will be on after the current instruction

    # Loop through all blocks that are on
    for old in core:
        old:Cuboid  # Block that was on before the instruction

        # Two cuboids intersect when the minimum coordinate of one is smaller
        # than the maximum coordinate of the other, for all 3 axes
        intersect = all(map(less_than_or_equal, new.min, old.max)) and all(map(less_than_or_equal, old.min, new.max))

        # If there is no overlap between the old and new blocks,
        # then the old block will still remain on regardless of the instruction
        if not intersect:
            new_core += [old]
            continue

        # Slice the blocks along the intersection plane formed by the instruction and the block that is on
        # (the parts of the blocks above all the six faces of the instruction will become new blocks,
        # while the intersection area is removed)
        
        # x-axis
        if old.min.x <= new.max.x <= old.max.x:  # Positive direction
            new_core += [Cuboid(Coordinate(new.max.x+1, old.min.y, old.min.z), Coordinate(old.max.x, old.max.y, old.max.z))]
            old = Cuboid(Coordinate(old.min.x, old.min.y, old.min.z), Coordinate(new.max.x, old.max.y, old.max.z))

        if old.min.x <= new.min.x <= old.max.x:  # Negative direction
            new_core += [Cuboid(Coordinate(old.min.x, old.min.y, old.min.z), Coordinate(new.min.x-1, old.max.y, old.max.z))]
            old = Cuboid(Coordinate(new.min.x, old.min.y, old.min.z), Coordinate(old.max.x, old.max.y, old.max.z))

        # y-axis
        if old.min.y <= new.max.y <= old.max.y:  # Positive direction
            new_core += [Cuboid(Coordinate(old.min.x, new.max.y+1, old.min.z), Coordinate(old.max.x, old.max.y, old.max.z))]
            old = Cuboid(Coordinate(old.min.x, old.min.y, old.min.z), Coordinate(old.max.x, new.max.y, old.max.z))

        if old.min.y <= new.min.y <= old.max.y:  # Negative direction
            new_core += [Cuboid(Coordinate(old.min.x, old.min.y, old.min.z), Coordinate(old.max.x, new.min.y-1, old.max.z))]
            old = Cuboid(Coordinate(old.min.x, new.min.y, old.min.z), Coordinate(old.max.x, old.max.y, old.max.z))

        # z-axis
        if old.min.z <= new.max.z <= old.max.z:  # Positive direction
            new_core += [Cuboid(Coordinate(old.min.x, old.min.y, new.max.z+1), Coordinate(old.max.x, old.max.y, old.max.z))]
            old = Cuboid(Coordinate(old.min.x, old.min.y, old.min.z), Coordinate(old.max.x, old.max.y, new.max.z))

        if old.min.z <= new.min.z <= old.max.z:  # Negative direction
            new_core += [Cuboid(Coordinate(old.min.x, old.min.y, old.min.z), Coordinate(old.max.x, old.max.y, new.min.z-1))]
            old = Cuboid(Coordinate(old.min.x, old.min.y, new.min.z), Coordinate(old.max.x, old.max.y, old.max.z))
    
    # Process the operation
    if turn == "on": new_core += [new]  # If the area is to be turned on, add the region of the instruction
    else: assert turn == "off"          # Ensure that there are no instructions besides "on" and "off"
    core = new_core                     # Replace the old list of "on" blocks by the new one

# Calculate the total number of blocks
volume = 0
for block in core:
    volume += (block.max.x - block.min.x + 1) * (block.max.y - block.min.y + 1) * (block.max.z - block.min.z + 1)

total_time = perf_counter() - start_time

print(f"Part 2: {volume} (took {total_time:.3f} seconds)")

Part 2: 1201259791805392 (took 2.931 seconds)


In [24]:
import math
class Scanner:
    def __init__(self, id_):
        self.points = []
        self.distances = []
        self.id_ = id_
        self.offset = [0, 0, 0]
        self.axis_sign = [1, 1, 1]
        self.axis_map = [0, 1, 2]

    def calculate_distances(self):
        self.distances = []
        # Create a matrix of distances
        for i in range(len(self.points)):
            p1 = self.points[i]
            distances = []

            for j in range(len(self.points)):
                if i == j:
                    continue
                p2 = self.points[j]
                dist = math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2 + (p2[2] - p1[2])**2)
                distances.append(dist)

            distances.sort()
            self.distances.append(distances)

    def find_overlapping(self, other: 'Scanner'):
        common_points = {}
        for i in range(len(self.distances)):
            for j in range(len(other.distances)):
                _c = 0
                if i in common_points:
                    break
                for k in range(12):
                    if self.distances[i][k] == other.distances[j][k]:
                        _c += 1
                if _c > 1:
                    # print(_c)
                    common_points[i] = j
        if len(common_points) < 1:
            # Apparently 12 points are always shared.
            return False

        # Determine axis switch. By default we assume complete alignment
        axis_map = [0, 1, 2]
        axis_sign = [1, 1, 1]
        offset = [None, None, None]

        # For all possibilities of rotated and flipped axes, try to determine
        # the offset.
        for i in range(3):
            # i - axis rotation
            for j in [1, -1]:

                _offset_x = []
                _offset_y = []
                _offset_z = []

                for key in common_points:
                    p1 = self.points[key]
                    p2 = other.points[common_points[key]]
                    _offset_x.append(p1[0] - j * p2[i])
                    _offset_y.append(p1[1] - j * p2[i])
                    _offset_z.append(p1[2] - j * p2[i])

                # When applying axis mapping, do not do both switcheroos!
                # Other final mapping will be wrong.
                if len(set(_offset_x)) == 1:
                    axis_map[0] = i
                    axis_sign[0] = j
                    offset[0] = _offset_x[0]
                if len(set(_offset_y)) == 1:
                    axis_map[1] = i
                    axis_sign[1] = j
                    offset[1] = _offset_y[0]
                if len(set(_offset_z)) == 1:
                    axis_map[2] = i
                    axis_sign[2] = j
                    offset[2] = _offset_z[0]
                # print(_offset_x, _offset_y, _offset_z)

        # If one of the axis is undetermined stop.
        if any([_ == None for _ in offset]):
            return False

        other.offset = offset
        other.axis_map = axis_map
        other.axis_sign = axis_sign
        other.align_points()
        return True

    def align_points(self):
        """Function that aligns the offset with the root scanner.
        """
        for i in range(len(self.points)):
            x, y, z = self.axis_map
            sx, sy, sz = self.axis_sign

            new_x = self.offset[0] + sx * self.points[i][x]
            new_y = self.offset[1] + sy * self.points[i][y]
            new_z = self.offset[2] + sz * self.points[i][z]

            self.points[i][0] = new_x
            self.points[i][1] = new_y
            self.points[i][2] = new_z


scanners = []
file = "day_19.dat"
with open(file, 'r') as f:
    current = None
    _c = 0
    for line in f:
        if line.startswith('---'):
            if current:
                current.points = points
                current.calculate_distances()
            current = Scanner(_c)
            _c += 1
            scanners.append(current)
            points = []
            continue
        if not line.strip():
            continue

        points.append([int(_) for _ in line.split(',')])

    current.points = points
    current.calculate_distances()

to_process = scanners[:]
processed = [scanners[0]]
to_process.pop(0)

while to_process:
    # pop the first one
    scanner = to_process.pop(0)

    for i, aligned in enumerate(processed):
        ok = aligned.find_overlapping(scanner)
        if ok:
            # print(f'Aligned to {i}')
            break
    if ok:
        processed.append(scanner)
        # for aligned in processed:
        #     print(aligned.points)
    else:
        to_process.append(scanner)

# Gather all the points and return the number of beacons
points = []
for scanner in processed:
    for point in scanner.points:
        if point in points:
            continue
        else:
            points.append(point)
print("Part 1. Number of beacons:", len(points))


max_distance_between_2scanners = 0
for i in range(len(processed)):
    for j in range(i, len(processed)):
        s1 = processed[i]
        s2 = processed[j]

        dist = abs(s1.offset[0] - s2.offset[0]) + abs(s1.offset[1] - s2.offset[1]) + abs(s1.offset[2] - s2.offset[2])

        if dist > max_distance_between_2scanners:
            max_distance_between_2scanners = dist
print("Part 2. Max manhattan distance between two scanners: ", max_distance_between_2scanners)

Part 1. Number of beacons: 385
Part 2. Max manhattan distance between two scanners:  10707


In [17]:
#day18
import itertools
import math
import re


def add(data):
    if " + " in data:
        data = f"[{data.split(' + ')[0]},{data.split(' + ')[1]}]"
    return data


def explode(data):
    offset = 0
    for p in re.findall("\[\d+,\d+\]", data):
        pair = re.search(re.escape(p), data[offset:])
        left_brackets = data[: pair.start() + offset].count("[")
        right_brackets = data[: pair.start() + offset].count("]")
        if left_brackets - right_brackets >= 4:
            x, y = pair.group()[1:-1].split(",")
            # split the string into two parts at the pair
            # flip left side around so we get the first num going backwards
            left = data[: pair.start() + offset][::-1]
            right = data[pair.end() + offset :]
            # look left
            search_left = re.search("\d+", left)
            if search_left:
                # need to find the rightmost match not the first
                amt = int(left[search_left.start() : search_left.end()][::-1]) + int(x)
                left = f"{left[:search_left.start()]}{str(amt)[::-1]}{left[search_left.end():]}"
            # look right
            search_right = re.search("\d+", right)
            if search_right:
                amt = int(right[search_right.start() : search_right.end()]) + int(y)
                right = (
                    f"{right[:search_right.start()]}{amt}{right[search_right.end():]}"
                )
            data = f"{left[::-1]}0{right}"
            break
        else:
            offset = pair.end() + offset
    return data


def split(data):
    dd = re.search("\d\d", data)
    if dd:
        left = data[: dd.start()]
        right = data[dd.end() :]
        left_digit = int(math.floor(int(dd.group()) / 2))
        right_digit = int(math.ceil(int(dd.group()) / 2))
        data = f"{left}[{left_digit},{right_digit}]{right}"
    return data


def reduce(data):
    exploded = explode(data)
    if exploded != data:
        return reduce(exploded)
    else:
        splitd = split(data)
        if splitd != data:
            return reduce(splitd)
        else:
            return splitd


def magnitude(data):
    while data.count(",") > 1:
        for p in re.findall("\[\d+,\d+\]", data):
            pair = re.search(re.escape(p), data)
            left_digit, right_digit = p[1:-1].split(",")
            data = f"{data[: pair.start()]}{int(left_digit) * 3 + int(right_digit) * 2}{data[pair.end() :]}"
    left_digit, right_digit = data[1:-1].split(",")
    return int(left_digit) * 3 + int(right_digit) * 2


# PART 1
data = open("day18.in").read().strip().split("\n")
p1 = list(data)
sum = ""
final_sum = ""
while p1:
    line1 = p1.pop(0)
    if not final_sum:
        line2 = p1.pop(0)
        final_sum = f"{line1} + {line2}"
    else:
        final_sum = f"{final_sum} + {line1}"
    final_sum = reduce(add(final_sum))
print(f"Part 1: {magnitude(final_sum)}")

# PART 2
p2 = list(data)
magnitudes = set()
pairs = list(itertools.permutations(p2, 2))
for pair in pairs:
    final_sum = reduce(add(f"{pair[0]} + {pair[1]}"))
    magnitudes.add(magnitude(final_sum))
print(f"Part 2: {max(magnitudes)}")


Part 1: 4057
Part 2: 4683


In [14]:
#day16
bs = bin(int('1'+open("16").read(),16))[3:]

def ps1(startbit):
    i = startbit # index into bs
    tv = int(bs[i:i+3],2) # total version
    ID = int(bs[i+3:i+6],2) # packet type ID
    i += 6
    if ID == 4: #literal value
        while True:
            i += 5
            if bs[i-5] == '0': #last value packet
                break
    else:
        if bs[i] == '0':
            endi = i + 16 + int(bs[i+1:i+16],2)
            i += 16
            while i < endi:
                i,v = ps1(i)
                tv += v
        else:
            np = int(bs[i+1:i+12],2)
            i += 12
            for _ in range(np):
                i,v = ps1(i)
                tv += v

    return i,tv

print("Total of version numbers:",ps1(0)[1])


### PART 2 ###

import math

op = [sum, math.prod, min, max,
      lambda ls: ls[0], # literal
      lambda ls: 1 if ls[0] > ls[1] else 0,  # gt
      lambda ls: 1 if ls[0] < ls[1] else 0,  # lt
      lambda ls: 1 if ls[0] == ls[1] else 0] # eq

def ps2(startbit):
    i = startbit # index into bs
    ID = int(bs[i+3:i+6],2) # packet type ID
    i += 6
    if ID == 4: #literal value
        vals = [0]
        while True:
            vals[0] = 16*vals[0] + int(bs[i+1:i+5],2)
            i += 5
            if bs[i-5] == '0': #last value packet
                break
    else:
        vals = []
        if bs[i] == '0': # subpacket length in bits
            endi = i + 16 + int(bs[i+1:i+16],2)
            i += 16
            while i < endi:
                i,v = ps2(i)
                vals.append(v)
        else:
            np = int(bs[i+1:i+12],2) # number of subpackets
            i += 12
            for _ in range(np):
                i,v = ps2(i)
                vals.append(v)

    return i,op[ID](vals)

print('Total value:',ps2(0)[1])

Total of version numbers: 957
Total value: 744953223228


In [10]:
#day 13
#!/usr/bin/python3

import sys
lines = open("13",'r').read().split('\n')[:-1]

points = list()
folds = list()
for l in lines:
    if (l.startswith('fold')):
        d,v = l[11:].split('=')
        folds.append((d,int(v)))
    elif (l != ''):
        x,y = l.split(',')
        points.append((int(x),int(y)))

def printPoints(points):
    maxY = max([p[1] for p in points])
    maxX = max([p[0] for p in points])
    print()
    for y in range(maxY+1):
        for x in range(maxX+1):
            if (x,y) in points:
                print('█',end='')
            else:
                print(' ',end='')
        print()
    print()

def foldPaper(points, fold):
    if (fold[0] == 'y'):
        for i in range(len(points)):
            if (points[i][1] > fold[1]):
                points[i] = (points[i][0], points[i][1] - 2 * (points[i][1] - fold[1]))
    elif (fold[0] == 'x'):
        for i in range(len(points)):
            if (points[i][0] > fold[1]):
                points[i] = (points[i][0] - 2 * (points[i][0] - fold[1]),points[i][1])
    return list(set(points))

for i in range(len(folds)):
    if (i == 1):
        print('Part 1:',len(points))
    points = foldPaper(points, folds[i])
print('Part 2:')
printPoints(points)

Part 1: 618
Part 2:

 ██  █    ███  ████ █  █ ████ █  █ █  █
█  █ █    █  █ █    █ █  █    █ █  █  █
█  █ █    █  █ ███  ██   ███  ██   █  █
████ █    ███  █    █ █  █    █ █  █  █
█  █ █    █ █  █    █ █  █    █ █  █  █
█  █ ████ █  █ ████ █  █ █    █  █  ██ 

